# Shmoo Plot Demo

In [139]:
# Equipment setup

import pyvisa
import time
import subprocess
import serial

# Establish connection to the power supply
rm = pyvisa.ResourceManager("@py")

#psu = rm.open_resource('TCPIP::169.254.58.10::gpib0,7::INSTR')                  ## for the Agilent power supply

psu = rm.open_resource('TCPIP::169.254.201.77::lan0::INSTR')                    ## for the Keysight power supply

clk = rm.open_resource('TCPIP::169.254.201.101::lan0::INSTR')

# Need to verify that this is the power supply. psu_verification should be 
# something like "Keysight Technologies,E3646A,0,X.X-Y.Y-Z.Z"
psu_verification = psu.query("*IDN?")
clk_verification = clk.query("*IDN?")

print("Power supply:", psu_verification)
print("Clock generator:", clk_verification)

# psu.write("*RST")  # Reset the power supply

start_voltage = 0.55

end_voltage = 0.65

step_size = 0.05  

start_clock = 20

end_clock = 40

step_size_clock = 20

# Loop through the voltage range and set the voltage
current_voltage = start_voltage 

current_clock = start_clock

psu.write("OUTP ON, (@2)")

# Only channel 2 works
psu.write("INST:SEL CH2")

# Set current to 100mA
psu.write("CURR 0.1")

# Start the clock at 20MHz
clk.write(":OUTP1:POS ON") # Initialize clk gen channel 1 output
clk.write(":FUNC:PATT")

measured_voltages = []
measured_currents = []

Power supply: Keysight Technologies,E36312A,MY57010726,1.0.4-1.0.0-1.04

Clock generator: Agilent Technologies, 81134A, DE42800131, Ver 2.9.3



#### Remember to reset the board!

In [140]:
while current_clock <= end_clock:
     # remake to adjust with the frequency
     subprocess.run(["make", "clean"], cwd="../../")

     with open("../../platform/bearly24/freq.h", "w") as f:
          f.write('''#ifndef __FREQ_H
                    #define __FREQ_H
                    ''')
          f.write(f'''#define MTIME_FREQ     {current_clock}000000
                    #define SYS_CLK_FREQ   {current_clock}000000
                    ''')
          f.write("#endif")

     subprocess.run(["make", "shmoo_bearly"], cwd="../../")

     clk.write(f":FREQ {current_clock}MHz")
     print(f"**Clock set to: {current_clock}MHz**") 


     while current_voltage <= end_voltage: # Set the voltage   
          psu.write(f"VOLT {current_voltage}")
          psu.write("OUTP ON, (@2)")

          voltage = psu.query("VOLT?") 
          print(f"**Voltage set to: {voltage}V**") 

          # test subprocess with simple hello
          with subprocess.Popen(["uart_tsi", "+tty=/dev/ttyUSB1", "+baudrate=921600", "../../build/shmoo_bench.elf"]) as proc:
               with serial.Serial('/dev/ttyUSB0', 115200) as ser:
                    while (ser.inWaiting() == 0):
                         ## measure the voltage + current
                         measured_voltages.append(psu.query("MEAS:VOLT?"))
                         measured_currents.append(psu.query("MEAS:CURR?"))
                         # print("hi")
                    line = ser.readline()
                    new_line = line.decode("utf-8")

                    with open('../../output.txt', 'a') as file:
                         file.write(f"Freq: {current_clock}MHz, Voltage: {current_voltage}V, Output: {new_line}")
                         proc.terminate()
                         proc.wait()
          print("Done: now reset during the sleep before next iteration")
          time.sleep(7)

          current_voltage += step_size
     current_voltage = start_voltage
     current_clock += step_size_clock

print("Finished")

psu.close()

current_clock = start_clock
current_voltage = start_voltage

rm -rf build
cmake -S ./ -B ./build/ -D CMAKE_TOOLCHAIN_FILE=./riscv-gcc.cmake -DCHIP=bearly24
-- The ASM compiler identification is GNU
-- Found assembler: /tools/C/ee290-fa24-2/.conda-env/riscv-tools/bin/riscv64-unknown-elf-gcc
-- The C compiler identification is GNU 13.2.0
-- The CXX compiler identification is GNU 13.2.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /tools/C/ee290-fa24-2/.conda-env/riscv-tools/bin/riscv64-unknown-elf-gcc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /tools/C/ee290-fa24-2/.conda-env/riscv-tools/bin/riscv64-unknown-elf-g++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Including HTIF to target
-- Including CLINT to target
-- ================ Glossy config ================
--  Building for CHIP: "bearly24"
--

/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/uart/uart.c: In function 'uart_receive':
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/uart/uart.c:35:78: warning: unused parameter 'timeout' [-Wunused-parameter]
   35 | Status uart_receive(UART_Type *UARTx, uint8_t *data, uint16_t size, uint32_t timeout) {
      |                                                                     ~~~~~~~~~^~~~~~~
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/uart/uart.c: In function 'uart_transmit':
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/uart/uart.c:50:85: warning: unused parameter 'timeout' [-Wunused-parameter]
   50 | Status uart_transmit(UART_Type *UARTx, const uint8_t *data, uint16_t size, uint32_t timeout) {
      |                                                                            ~~~~~~~~~^~~~~~~


[  5%] Linking C static library libuart.a
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[  5%] Built target uart
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[  8%] Building C object driver/intel/pll/CMakeFiles/intel-pll.dir/pll.c.obj
[ 11%] Linking C static library libintel-pll.a
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[ 11%] Built target intel-pll
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[ 13%] Building C object driver/rocket-chip/clint/CMakeFiles/clint.dir/clint.c.obj
[ 16%] Linking C static library libclint.a
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[

/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/i2c/i2c.c: In function 'i2c_init':
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/i2c/i2c.c:13:45: warning: unused parameter 'I2C_init' [-Wunused-parameter]
   13 | void i2c_init(I2C_Type *I2Cx, I2C_InitType *I2C_init) {
      |                               ~~~~~~~~~~~~~~^~~~~~~~
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/i2c/i2c.c: In function 'i2c_wait_for_flag':
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/i2c/i2c.c:30:9: warning: implicit declaration of function 'CLINT_getTime' [-Wimplicit-function-declaration]
   30 |     if (CLINT_getTime() > (timestart + timeout)) {
      |         ^~~~~~~~~~~~~
/tools/C/tedkim7/sp24-Baremetal-IDE/driver/rocket-chip-blocks/i2c/i2c.c:30:25: warning: comparison of integer expressions of different signedness: 'int' and 'uint32_t' {aka 'unsigned int'} [-Wsign-compare]
   30 |     if (CLINT_getTime() > (timestart + timeout)) {
      |    

[ 27%] Linking C static library libi2c.a
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[ 27%] Built target i2c
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[ 30%] Building C object driver/rocket-chip-blocks/spi/CMakeFiles/spi.dir/spi.c.obj


...

[ 97%] Building C object CMakeFiles/shmoo_bench.dir/shmoo_bench/hello/src/main.c.obj


/tools/C/tedkim7/sp24-Baremetal-IDE/shmoo_bench/hello/src/main.c: In function 'app_main':
/tools/C/tedkim7/sp24-Baremetal-IDE/shmoo_bench/hello/src/main.c:71:13: warning: unused variable 'two' [-Wunused-variable]
   71 |     uint8_t two = 1+1;
      |             ^~~
/tools/C/tedkim7/sp24-Baremetal-IDE/shmoo_bench/hello/src/main.c: In function 'main':
/tools/C/tedkim7/sp24-Baremetal-IDE/shmoo_bench/hello/src/main.c:80:14: warning: unused parameter 'argc' [-Wunused-parameter]
   80 | int main(int argc, char **argv) {
      |          ~~~~^~~~
/tools/C/tedkim7/sp24-Baremetal-IDE/shmoo_bench/hello/src/main.c:80:27: warning: unused parameter 'argv' [-Wunused-parameter]
   80 | int main(int argc, char **argv) {
      |                    ~~~~~~~^~~~


[100%] Linking C executable shmoo_bench.elf


/bwrcq/C/ee290-fa24-2/.conda-env/riscv-tools/bin/../lib/gcc/riscv64-unknown-elf/13.2.0/../../../../riscv64-unknown-elf/bin/ld: warning: shmoo_bench.elf has a LOAD segment with RWX permissions


gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[100%] Built target shmoo_bench
gmake[3]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[2]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[1]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'


In [136]:
print(measured_voltages)
print(measured_currents)

['+5.48995000E-01\n', '+5.48995000E-01\n', '+5.48995000E-01\n', '+5.48995000E-01\n', '+5.99905000E-01\n', '+5.99905000E-01\n', '+5.99070000E-01\n', '+5.99905000E-01\n', '+5.49830000E-01\n', '+5.48995000E-01\n', '+5.48995000E-01\n', '+5.48995000E-01\n', '+5.99905000E-01\n', '+5.99905000E-01\n', '+5.99070000E-01\n', '+5.99905000E-01\n']
['+4.61500000E-03\n', '+6.12000000E-03\n', '+6.11800000E-03\n', '+6.52400000E-03\n', '+6.37500000E-03\n', '+6.37400000E-03\n', '+6.37500000E-03\n', '+6.82300000E-03\n', '+4.19200000E-03\n', '+7.68600000E-03\n', '+7.68700000E-03\n', '+8.53800000E-03\n', '+8.13900000E-03\n', '+8.13700000E-03\n', '+8.13500000E-03\n', '+9.02100000E-03\n']


16